## Training a neural network to classify images of MFCCs

In [1]:
#ON WINDOWS SYSTEM
""""
import os

# get parent directory
parent_dir = os.path.dirname(os.getcwd())

# get the list of csv files in the folder
directory = parent_dir + "\\datasets\\"

csv_files_ita = [directory + "\\ita\\" + f for f in os.listdir(directory + "ita\\") if f.endswith('.csv')]
csv_files_eng = [directory + "\\eng\\" + f for f in os.listdir(directory + "eng\\") if f.endswith('.csv')]

print(len(csv_files_ita), len(csv_files_eng))
"""

'"\nimport os\n\n# get parent directory\nparent_dir = os.path.dirname(os.getcwd())\n\n# get the list of csv files in the folder\ndirectory = parent_dir + "\\datasets\\"\n\ncsv_files_ita = [directory + "\\ita\\" + f for f in os.listdir(directory + "ita\\") if f.endswith(\'.csv\')]\ncsv_files_eng = [directory + "\\eng\\" + f for f in os.listdir(directory + "eng\\") if f.endswith(\'.csv\')]\n\nprint(len(csv_files_ita), len(csv_files_eng))\n'

In [3]:
#ON WINDOWS SYSTEM

import os

current_dir = os.getcwd()
directory = os.path.dirname(current_dir) + "/datasets/"
csv_files_ita = [directory + "/ita/" + f for f in os.listdir(directory + "ita/") if f.endswith('.csv')]
csv_files_eng = [directory + "/eng/" + f for f in os.listdir(directory + "eng/") if f.endswith('.csv')]

print(len(csv_files_ita), len(csv_files_eng))


425 450


Test import of csv datasets into tensorflow datasets

import every csv file as a single matrix with one label associated

In [4]:
import tensorflow as tf

# Print TensorFlow version

print("TensorFlow version:", tf.__version__)

 

print(tf.config.list_physical_devices('GPU'))

# Check if GPU is available and being used



TensorFlow version: 2.4.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
import numpy as np

# read csv file and store the values into a numpy matrix
data_array = np.genfromtxt(csv_files_ita[0], delimiter=',', dtype=np.int8)
print("Loaded data array shape:", data_array.shape)

label = "ita"
test = [[1, 2], [3, 4]]

# Create a TensorFlow dataset
tf_dataset_matrix = tf.data.Dataset.from_tensor_slices([data_array])
tf_dataset_label = tf.data.Dataset.from_tensor_slices([label])

tf_dataset = tf.data.Dataset.zip((tf_dataset_matrix, tf_dataset_label))

for element in tf_dataset:
    print(element)

Loaded data array shape: (1247, 12)
(<tf.Tensor: shape=(1247, 12), dtype=int8, numpy=
array([[127,   4, -96, ..., -46, -34,  30],
       [118,  13, -94, ..., -32, -60,  43],
       [108,  30, -94, ...,  -8, -94,  39],
       ...,
       [-33, -16, -25, ...,  50,  20,   6],
       [-30,  12, -37, ...,  39,  29, -31],
       [-26,  45, -40, ...,   5,  56, -29]], dtype=int8)>, <tf.Tensor: shape=(), dtype=string, numpy=b'ita'>)


2023-08-11 17:28:24.116032: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 17:28:24.118881: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-08-11 17:28:24.119674: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-11 17:28:24.119737: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 119

In [6]:
# create empty pandas dataframe
dataset = []
labels = []

# read csv files into dataframe
for file in csv_files_ita:
    data_array = np.genfromtxt(file, delimiter=',', dtype=np.int8)
    dataset.append(data_array)
    labels.append("ita")

for file in csv_files_eng:
    data_array = np.genfromtxt(file, delimiter=',', dtype=np.int8)
    dataset.append(data_array)
    labels.append("eng")

print("dataset size: ", len(dataset))
print("labels size: ", len(labels))


dataset size:  875
labels size:  875


In [7]:
classes = ["ita", "eng"]

# Create a mapping from class names to integer labels
class_to_index = {class_name: index for index, class_name in enumerate(classes)}

# Convert labels to integer labels using the mapping
integer_labels = np.array([class_to_index[label] for label in labels], dtype=np.int8)

labels_one_hot = tf.keras.utils.to_categorical(integer_labels, num_classes=2) # one hot encoding

print(labels_one_hot)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [10]:
# train - validation split of tensorflow dataset
from sklearn.model_selection import train_test_split

train_features, val_features, train_labels, val_labels = train_test_split(dataset, labels_one_hot, test_size=0.2, random_state=42) 
print(val_features[0].dtype)

val_features = tf.reshape(val_features, (-1, 1247, 12, 1))
train_features = tf.reshape(train_features, (-1, 1247, 12, 1))


# create tensorflow dataset from numpy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels))

# shuffle and batch
train_dataset = train_dataset.shuffle(len(train_features))

val_dataset = val_dataset.batch(32)
train_dataset = train_dataset.batch(32)

int8


In [11]:
for image_batch, labels_batch in train_dataset:
	print(image_batch.shape)
	print(labels_batch.shape)
	break


(32, 1247, 12, 1)
(32, 2)


In [12]:
from tensorflow.keras import layers, models
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)
# Create a basic CNN model
model = models.Sequential([
    #layers.Reshape(( 1247, 12), input_shape=(1247, 12)),
	layers.Conv2D(filters=64, kernel_size=(5, 1), activation='relu', input_shape=(1247, 12, 1)),
	layers.MaxPooling2D(pool_size=(2, 1)),
    layers.Conv2D(filters=64, kernel_size=(5, 1), activation='relu'),
	layers.MaxPooling2D(pool_size=(2, 1)),
    layers.Conv2D(filters=32, kernel_size=(3, 1), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 1)),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.GlobalAveragePooling2D(), 
	layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
	layers.Dense(2, activation='softmax')  # Two classes
])

model.summary()

# Compile the model
model.compile(optimizer='adam',
			  loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
			  metrics=['accuracy'])

parent_dir = os.path.dirname(os.getcwd())
filepath = parent_dir + "\\models\\"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy',save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_loss', mode='min', patience=10)

callbacks_list = [checkpoint, es]

# Train the model
model.fit(x=train_dataset, epochs=50, callbacks=callbacks_list, batch_size=32, 
          validation_data=val_dataset)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1243, 12, 64)      384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 621, 12, 64)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 617, 12, 64)       20544     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 308, 12, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 306, 12, 32)       6176      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 153, 12, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 151, 10, 32)       9

2023-08-11 17:30:38.059922: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-08-11 17:30:38.069378: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400005000 Hz
2023-08-11 17:30:38.402324: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-08-11 17:30:39.404955: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-08-11 17:30:44.596945: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2023-08-11 17:30:44.967317: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


22/22 [==============================] - 80s 2s/step - loss: 1.0930 - accuracy: 0.5137 - val_loss: 0.6565 - val_accuracy: 0.6914


2023-08-11 17:31:57.252662: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/claudio/EmbeddedAI/Spoken_Language_Recognition_Tensorflow_Embedded\models\/assets
Epoch 2/50
22/22 [==============================] - 2s 72ms/step - loss: 0.6672 - accuracy: 0.6128 - val_loss: 0.6649 - val_accuracy: 0.5371
Epoch 3/50
22/22 [==============================] - 2s 70ms/step - loss: 0.6778 - accuracy: 0.5747 - val_loss: 0.6985 - val_accuracy: 0.5429
Epoch 4/50
22/22 [==============================] - 2s 71ms/step - loss: 0.6824 - accuracy: 0.6034 - val_loss: 0.6517 - val_accuracy: 0.5600
Epoch 5/50
22/22 [==============================] - 2s 71ms/step - loss: 0.7074 - accuracy: 0.5400 - val_loss: 0.7191 - val_accuracy: 0.4914
Epoch 6/50
22/22 [==============================] - 2s 70ms/step - loss: 0.6862 - accuracy: 0.5576 - val_loss: 0.6401 - val_accuracy: 0.6457
Epoch 7/50
22/22 [==============================] - 2s 70ms/step - loss: 0.6410 - accuracy: 0.6140 - val_loss: 0.6068 - val_accuracy: 0.7086
INFO:tensorflow:Assets written 

In [28]:
filepath = parent_dir + "\\model_lite\\"
model.save(filepath +  "CNN_model_h5")

INFO:tensorflow:Assets written to: C:\Users\HP\Documents\GitHub\Spoken_Language_Recognition_Tensorflow_Embedded\model_lite\CNN_model_h5\assets


INFO:tensorflow:Assets written to: C:\Users\HP\Documents\GitHub\Spoken_Language_Recognition_Tensorflow_Embedded\model_lite\CNN_model_h5\assets


In [50]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(filepath + "CNN_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    #tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]

converter.experimental_enable_resource_variables = True

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(filepath + "CNN_model.tflite", 'wb') as f:
	f.write(tflite_model)

In [51]:
model_path = filepath + "CNN_model.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_conv2d_49_input:0', 'index': 0, 'shape': array([   1, 1247,   12,    1]), 'shape_signature': array([  -1, 1247,   12,    1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 28, 'shape': array([1, 2]), 'shape_signature': array([-1,  2]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [52]:
# Assuming single input and output tensors.
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

print(input_shape)
print(output_shape)

[   1 1247   12    1]
[1 2]


In [77]:
random_index = np.random.randint(0, len(dataset))

print(dataset[0])
# Select the random data point using the random index
random_data_point = dataset[random_index]
random_label = labels_one_hot[random_index]
# Convert the random data point from int8 to float32
input_data_matrix = random_data_point.astype(np.float32)
batch_size = 1

input_data_matrix = tf.reshape(input_data_matrix, (-1, 1247, 12, 1))


[[-89  32  76 ...  35  63  49]
 [-84  18  57 ... -11  61  22]
 [-83  14  66 ... -32  56  20]
 ...
 [ -6  87  22 ... -14  14   5]
 [ 11  71  -4 ...  -7   1 -16]
 [ 32  69 -33 ... -17   0   2]]


In [78]:
# Set input data to the interpreter.
interpreter.set_tensor(input_details[0]['index'], input_data_matrix)

# Run inference.
interpreter.invoke()

# Get output data from the interpreter.
output_data = interpreter.get_tensor(output_details[0]['index'])

In [79]:
# Process output data.
# For example, if your output is classification probabilities:
predicted_class = np.argmax(output_data)
print("Predicted class:", predicted_class)

Predicted class: 1


In [80]:
print(random_label)

[0. 1.]
